### AB Model from spec

Package installation

In [1]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [2]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
import timeit

import logging
os.environ["PATH"] += os.pathsep + '/Users/isaacao/homebrew/bin' # for graphviz

Model component importation

In [3]:
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM, LSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_charging, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered

State diagram

### Model

Model parameters

In [4]:
ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
no_evs = 2
# no of css must be fixed
no_css = 5

Run Model

In [5]:
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css)
for i in range(ticks):
    model_run.step()


EV info: ID: 0, destination name: City C, journey type: InterUrban, max_battery: 62, speed: 20, State: Idle.
EV info (Cont'd): Start time: 8, distance goal: 150, energy consumption rate: 0.5, charge prop 0.4, location: City_D.

EV info: ID: 1, destination name: work, journey type: Urban, max_battery: 54, speed: 10, State: Idle.
EV info (Cont'd): Start time: 12, distance goal: 50, energy consumption rate: 0.2, charge prop 0.4, location: City_D.

CP info: ID: 2, initialized. Charge rate: 7.5 kW.

CP info: ID: 3, initialized. Charge rate: 7.5 kW.

CP info: ID: 4, initialized. Charge rate: 7.5 kW.

CP info: ID: 5, initialized. Charge rate: 7.5 kW.

CP info: ID: 6, initialized. Charge rate: 7.5 kW.
Charging Station: 2 is at checkpoint: 40 miles.
Charging Station: 3 is at checkpoint: 80 miles.
Charging Station: 4 is at checkpoint: 120 miles.
Charging Station: 5 is at checkpoint: 160 miles.
Charging Station: 6 is at checkpoint: 200 miles.

Model initialised. 2 EVs and 5 Charging Points. Simu

In [6]:
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

    EVs Charging  EVs Activated  EVs Travelling  EVs Queued EVs Charge Level  \
0              0              2               0           0         [62, 54]   
1              0              2               0           0         [62, 54]   
2              0              2               0           0         [62, 54]   
3              0              2               0           0         [62, 54]   
4              0              2               0           0         [62, 54]   
5              0              2               0           0         [62, 54]   
6              0              2               0           0         [62, 54]   
7              0              2               0           0         [62, 54]   
8              0              2               1           0       [52.0, 54]   
9              0              2               1           0       [42.0, 54]   
10             0              2               1           0       [32.0, 54]   
11             0              2         

In [ ]:
a = 26 % 24
print(a) 

Export results to CSV file

In [7]:
model_run.datacollector.get_model_vars_dataframe().to_csv('01_03_5EV_agent_model_output.csv')

In [ ]:
# def unpack_datacollector(dc):
#     """
#     Unpacks the data from a DataCollector instance in Mesa.
    
#     Args:
#         dc (DataCollector): A DataCollector instance in Mesa.
    
#     Returns:
#         A dictionary with the keys as the variable names and the values as the lists of data for each variable.
#     """
#     data = {}
#     for varname, var in dc.model_vars.items():
#         data[varname] = var.get_data()
#     return data

### Data preprocessing

Helper functions

In [18]:
# def split_column_values(df, col_name):
#     """
#     Splits the values in a column by commas and creates a new column for each value.

#     Args:
#         df (pandas.DataFrame): A pandas DataFrame.
#         col_name (str): The name of the column to split.

#     Returns:
#         A pandas DataFrame with additional columns for each value in the input column.
#     """
#     # Get the unique values in the column
#     unique_values = set(df[col_name].str.cat(sep=',').split(','))

#     # Create a new column for each unique value
#     for value in unique_values:
#         df[value] = df[col_name].str.contains(value).astype(int)

#     # Drop the original column
#     df.drop(col_name, axis=1, inplace=True)

#     return df

In [19]:
data = pd.read_csv('01_03_5EV_agent_model_output.csv')

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 14 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Unnamed: 0                              48 non-null     int64  
 1   EVs Charging                            48 non-null     int64  
 2   EVs Activated                           48 non-null     int64  
 3   EVs Travelling                          48 non-null     int64  
 4   EVs Queued                              48 non-null     int64  
 5   EVs Charge Level                        48 non-null     object 
 6   EVs Currently charging                  48 non-null     int64  
 7   EVs Not Idle                            48 non-null     int64  
 8   EOD Battery SOC                         48 non-null     object 
 9   EVs Destinations                        48 non-null     object 
 10  EVs at Charging Station - S             48 non-null     int64  


In [21]:
# a = data['EOD Battery SOC'].str.split(',').apply(pd.Series, 1).stack()
# a.index = a.index.droplevel(-1) # to line up with df's index
# a.name = 'EOD_EVs_SOCs' # needs a name to join
# del data['EOD_EVs_SOCs']
# data = data.join(a)
a = split_column_values(data, 'EOD Battery SOC')
print(a)


    Unnamed: 0  EVs Charging  EVs Activated  EVs Travelling  EVs Queued  \
0            0             0              2               0           0   
1            1             0              2               0           0   
2            2             0              2               0           0   
3            3             0              2               0           0   
4            4             0              2               0           0   
5            5             0              2               0           0   
6            6             0              2               0           0   
7            7             0              2               0           0   
8            8             0              2               1           0   
9            9             0              2               1           0   
10          10             0              2               1           0   
11          11             0              2               1           0   
12          12           

### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

### Scrapbook